# Washington State EV Adoption & Charger Forecast  
## Full End‑to‑End Pipeline 


**Outputs created by this notebook**
- County‑level charger baseline (Excel)
- 2024 EV / Population / Adoption baseline (Excel)
- Monte‑Carlo‑filled forecast (Excel)
- Statewide TOTAL row (Excel)
- Diagnostic charts (PNG)
- Baseline‑anchored best‑fit equations + 2050 values (Excel + PNG)

All equations are expressed in **(x − 2024)**.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from sklearn.metrics import r2_score
from scipy.optimize import curve_fit


## STEP 1 — Supercharger raw data → county‑level charge points


In [ ]:
super_path = "/Users/judycheng/Desktop/supercharger in washington state.xls"
output_super_summary = "/Users/judycheng/Desktop/supercharger_by_county_summary.xlsx"

df = pd.read_excel(super_path)

df_wa = df[df["State"] == "Washington"]

counts = (
    df_wa.groupby("County")["County"]
    .count()
    .reset_index(name="Supercharger_Count")
)

counts["County"] = counts["County"].str.replace(" County", "", regex=False)
counts = counts.sort_values("Supercharger_Count", ascending=False)

counts.to_excel(output_super_summary, index=False)

print("Exported:", output_super_summary)


## STEP 2 — Build 2024 baseline (Population + EVs + Chargers)


In [ ]:
residents_path = "/Users/judycheng/Desktop/Population 2024 age 25 to 59.xlsx"
ev_path = "/Users/judycheng/Desktop/coordinates_output.xlsm"
output_baseline = "/Users/judycheng/Desktop/wa_county_ev_forecast_baseline_2024_to_2050.xlsx"

residents_df = pd.read_excel(residents_path)
residents_df["County"] = residents_df["County"].str.replace(" County", "", regex=False)

pop_df = residents_df[["County", "total"]].rename(columns={"total": "Pop_2024"})

super_df = pd.read_excel(output_super_summary)
super_df = super_df.rename(columns={"Supercharger_Count": "Superchargers_2024"})

ev_df = pd.read_excel(ev_path)
ev_df["County"] = ev_df["County"].str.replace(" County", "", regex=False)

ev_counts = (
    ev_df.groupby("County")["County"]
    .count()
    .reset_index(name="EVs_2024")
)

base = pop_df.merge(super_df, on="County", how="left")
base = base.merge(ev_counts, on="County", how="left")

base = base.fillna(0)
base["Adoption_2024"] = base["EVs_2024"] / base["Pop_2024"]

for y in range(2025, 2051):
    base[f"Superchargers_{y}"] = pd.NA
    base[f"EVs_{y}"] = pd.NA
    base[f"Adoption_{y}"] = pd.NA

base.to_excel(output_baseline, index=False)
print("Exported:", output_baseline)


## STEP 3 — Monte‑Carlo template injection + TOTAL row


In [ ]:
input_baseline = output_baseline
output_mc = "/Users/judycheng/Desktop/wa_county_ev_forecast_filled_mc.xlsx"

king_path   = "/Users/judycheng/Desktop/king_county_ev_projection_mc_monotonic.xlsx"
pierce_path = "/Users/judycheng/Desktop/pierce_county_ev_projection_mc_monotonic.xlsx"
kitsap_path = "/Users/judycheng/Desktop/kitsap_county_ev_projection_mc_monotonic.xlsx"
chelan_path = "/Users/judycheng/Desktop/chelan_county_ev_projection_mc_monotonic.xlsx"

def load_mc(path):
    mc = pd.read_excel(path, sheet_name="Forecast")
    mc = mc.rename(columns={mc.columns[0]: "Year"}).set_index("Year")
    return mc[["Forecast_Chargers", "Forecast_Adoption_P50"]]

king_mc   = load_mc(king_path)
pierce_mc = load_mc(pierce_path)
kitsap_mc = load_mc(kitsap_path)
chelan_mc = load_mc(chelan_path)

def select_template(pop):
    if pop > 1_000_000:
        return king_mc
    elif pop > 130_000:
        return pierce_mc
    elif pop > 34_000:
        return kitsap_mc
    else:
        return chelan_mc

df = pd.read_excel(input_baseline)

YEARS = range(2025, 2051)

for idx, row in df.iterrows():
    template = select_template(row["Pop_2024"])
    for y in YEARS:
        df.at[idx, f"Adoption_{y}"] = template.loc[y, "Forecast_Adoption_P50"]
        df.at[idx, f"Superchargers_{y}"] = template.loc[y, "Forecast_Chargers"]
        df.at[idx, f"EVs_{y}"] = df.at[idx, f"Adoption_{y}"] * row["Pop_2024"]

# TOTAL row
total_pop = df["Pop_2024"].sum()
total = {"County": "TOTAL", "Pop_2024": total_pop}

for y in YEARS:
    total[f"EVs_{y}"] = df[f"EVs_{y}"].sum()
    total[f"Superchargers_{y}"] = df[f"Superchargers_{y}"].sum()
    total[f"Adoption_{y}"] = (df[f"Adoption_{y}"] * df["Pop_2024"]).sum() / total_pop

df = pd.concat([df, pd.DataFrame([total])], ignore_index=True)
df.to_excel(output_mc, index=False)

print("Exported:", output_mc)


## STEP 4 — Baseline‑anchored curve fitting + charts + equations


In [ ]:
input_path = output_mc
desktop = os.path.join(os.path.expanduser("~"), "Desktop")
chart_folder = os.path.join(desktop, "county_charts_baseline_anchored")
os.makedirs(chart_folder, exist_ok=True)

output_eq = os.path.join(desktop, "wa_county_baseline_anchored_equations.xlsx")

YEARS = np.arange(2024, 2051)
T = YEARS - 2024
T_FIT = T[1:]

def lin_anchor(t, a, y0): return a*t + y0
def quad_anchor(t, a, b, y0): return a*t**2 + b*t + y0
def cubic_anchor(t, a, b, c, y0): return a*t**3 + b*t**2 + c*t + y0

def sigmoid(z): return 1/(1+np.exp(-z))
def logistic_anchor(t, k, t0, y0, yT, tT=26):
    s0 = sigmoid(k*(0-t0))
    sT = sigmoid(k*(tT-t0))
    L = (yT-y0)/(sT-s0)
    return y0 + L*(sigmoid(k*(t-t0))-s0)

df = pd.read_excel(input_path)
rows = []

for _, r in df.iterrows():
    county = "Statewide" if r["County"] == "TOTAL" else r["County"]

    sc = np.array([r[f"Superchargers_{y}"] for y in YEARS])
    ad = np.array([r[f"Adoption_{y}"] for y in YEARS])

    sc0, scT = sc[0], sc[-1]
    ad0, adT = ad[0], ad[-1]

    fn = lambda t,a: lin_anchor(t,a,sc0)
    p,_ = curve_fit(fn,T_FIT,sc[1:])
    sc_hat = fn(T,*p)

    fn2 = lambda t,k,t0: logistic_anchor(t,k,t0,ad0,adT)
    p2,_ = curve_fit(fn2,T_FIT,ad[1:])
    ad_hat = fn2(T,*p2)

    rows.append({
        "County": county,
        "Supercharger_2050": sc_hat[-1],
        "Adoption_2050": ad_hat[-1]
    })

    plt.figure(figsize=(10,6))
    plt.plot(T, sc_hat, label="Charge Points")
    plt.plot(T, ad_hat, "--", label="Adoption")
    plt.title(county)
    plt.legend()
    plt.savefig(os.path.join(chart_folder, f"{county}.png"))
    plt.close()

pd.DataFrame(rows).to_excel(output_eq, index=False)

print("Charts:", chart_folder)
print("Equations:", output_eq)
